In [9]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [2]:
# Prepare the input and output data from the left/right hand seaquences
def prepare(data):
    data_input = data[:-1]
    data_output = data[1:]
    return data_input, data_output

In [3]:
#prepare the model
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        75,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(75, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(80))
    model.add(Dense(85))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [4]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]
    model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)

In [5]:
def train_network():
    """ Train a Neural Network to generate music """
    network_input, network_output = prepare(data)
    model = create_network(network_input, left.shape[1])
    train(model, network_input, network_output)

In [6]:
def read(filepath):
    return np.genfromtxt(filepath, delimiter=',')

In [7]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [10]:
data = read('left.csv')
network_input, network_output = prepare(data)

In [15]:
np.reshape(network_input, (len(network_input), 100, 90))

ValueError: cannot reshape array of size 174510 into shape (1939,100,90)

In [18]:
new = np.reshape(network_input, (network_input.shape[0],1,network_input.shape[1]))

In [19]:
new.shape

(1939, 1, 90)

In [29]:
import music21 as ms
from encoder_decoder import encode

def combine_save(left, rights, filepath, save):
    """
    Input: left - the stream of notes played by the 
                pianist with his left hand
            right - the stream of notes played by the 
                pianist with his right hand
            filepath - path to the file in which o/p
                midi stream is to be stored
    Output: N/A

    The combines the streams from the left stream and right 
    stream and outputs a single stream object
    """
    sc = ms.stream.Stream()
    sc.append(right)
    sc.append(left)
    left.offset = 0.0
    if save:
        sc.write('midi', filepath)
    return encode(sc)

In [33]:
right = ms.converter.parse('../bach_846-right_hand-decoded.mid')
left = ms.converter.parse('../bach_846-left_hand-decoded.mid')


In [26]:
sc = ms.stream.Stream()
sc.append(right)
sc.append(left)
left.offset = 0.0

In [49]:
sc = ms.stream.Stream()
sc = ms.stream.Stream()
sc.append(ms.instrument.Piano())
sc.append(ms.tempo.MetronomeMark(number=74))
sc.append(ms.key.Key("C"))
sc.append(ms.meter.TimeSignature())

mini = 0
for l_nt in left[0]:
    if type(l_nt) == ms.note.Note or type(l_nt) == ms.note.Rest or type(l_nt) == ms.chord.Chord:
        sc.append(l_nt)
        for r_nt in right[0]:
            if type(r_nt) == ms.note.Note or type(r_nt) == ms.note.Rest or type(r_nt) == ms.chord.Chord:
                if r_nt.offset <= l_nt.offset and r_nt.offset > mini:
                    try:
                        sc.append(l_nt)
                    except:
                        pass
    mini = int(r_nt.offset)


In [50]:
sc.write('midi', 'combine.mid')

'combine.mid'

In [51]:
from keras.layers import Activation